In [1]:
import tensorflow as tf

def analyze_ops_simple(model_path):
    """简单分析 TFLite 模型算子"""
    # 使用 tflite_runtime 或 tensorflow
    interpreter = tf.lite.Interpreter(model_path=model_path)
    
    # 加载模型信息
    with open(model_path, 'rb') as f:
        model_content = f.read()
    
    # 解析模型（需要安装 flatbuffers）
    try:
        import tflite
        from tflite.Model import Model
        from tflite.BuiltinOperator import BuiltinOperator
        
        model = Model.Model.GetRootAs(model_content, 0)
        
        # 获取所有算子
        used_ops = set()
        subgraph = model.Subgraphs(0)
        
        for i in range(subgraph.OperatorsLength()):
            op = subgraph.Operators(i)
            op_code_idx = op.OpcodeIndex()
            op_code = model.OperatorCodes(op_code_idx)
            
            builtin_code = op_code.BuiltinCode()
            if builtin_code >= 0:
                op_name = BuiltinOperator.BuiltinOperator().Name(builtin_code)
            else:
                op_name = f"CUSTOM_{op_code.CustomCode().decode('utf-8')}"
            
            used_ops.add(op_name)
        
        return sorted(list(used_ops))
    except ImportError:
        print("需要安装 flatbuffers: pip install flatbuffers")
        return []

# 分析你的模型
ops = analyze_ops_simple("models_mobilenet_v1_96/mobilenet_v1_96_int8.tflite")
print("模型使用的算子:")
for op in ops:
    print(f"  {op}")

2025-10-23 22:27:52.212486: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-23 22:27:52.288803: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-23 22:27:53.420463: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


需要安装 flatbuffers: pip install flatbuffers
模型使用的算子:


/home/melon/miniconda3/envs/embedded_ai_311/lib/python3.11/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
